In [1]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from qaoa_and_tdvp import QAOAResult
import math
from scipy.interpolate import griddata
import numpy as np
from qaoa_and_tdvp import QAOA

In [2]:
n=4

In [3]:
with open(f'.//results//data.p', 'rb') as file:
    data = pickle.load(file)
data.head(10)

instance  \
n p i                                             
3 1 0  <MaxCut.MaxCut object at 0x7f418e2077c0>   
    1  <MaxCut.MaxCut object at 0x7f418e206380>   
  2 0  <MaxCut.MaxCut object at 0x7f418e2077c0>   
    1  <MaxCut.MaxCut object at 0x7f418e206380>   
4 1 0  <MaxCut.MaxCut object at 0x7f41208ffa90>   
    1  <MaxCut.MaxCut object at 0x7f41208ffd30>   
    2  <MaxCut.MaxCut object at 0x7f41208fffd0>   
    3  <MaxCut.MaxCut object at 0x7f41205602b0>   
    4  <MaxCut.MaxCut object at 0x7f4120560550>   
    5  <MaxCut.MaxCut object at 0x7f41205607f0>   

                                                    tdvp  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
    1  instance                                    <M...   
    2  instance                                    <M...   
    3  instance                                    <M...   
    4  instance                                    <M...   
    5  instance                                    <M...   

                                                   scipy  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
    1  instance                                    <M...   
    2  instance                                    <M...   
    3  instance                                    <M...   
    4  instance                                    <M...   
    5  instance                                    <M...   

                                        gradient_descent  
n p i                                                     
3 1 0  instance                                    <M...  
    1  instance                                    <M...  
  2 0  instance                                    <M...  
    1  instance                                    <M...  
4 1 0  instance                                    <M...  
    1  instance                                    <M...  
    2  instance                                    <M...  
    3  instance                                    <M...  
    4  instance                                    <M...  
    5  instance                                    <M...

In [4]:
with open(f'.//results//stats.p', 'rb') as file:
    stats = pickle.load(file)
stats.head(10)

expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 tdvp             1                        0.911088  0.073714  0.858965   
                   2                        0.930390  0.063845  0.885245   
  scipy            1                        0.778976  0.112656  0.699316   
                   2                        0.930466  0.058679  0.888974   
  gradient_descent 1                        0.911161  0.073687  0.859056   
                   2                        0.942691  0.046450  0.909846   
4 tdvp             1                        0.555697  0.320149  0.133846   
                   2                        0.914538  0.023949  0.873142   
                   3                        0.926812  0.027653  0.895857   
                   4                        0.943432  0.049538  0.869999   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 tdvp             1  0.963212            0.869012  0.145009  0.766475   
                   2  0.975536            0.931829  0.071453  0.881304   
  scipy            1  0.858635            0.766332  0.006235  0.761923   
                   2  0.971958            0.893380  0.117401  0.810365   
  gradient_descent 1  0.963266            0.867664  0.146380  0.764157   
                   2  0.975536            0.912558  0.098755  0.842728   
4 tdvp             1  0.963225            0.464344  0.373130  0.019871   
                   2  0.947408            0.895209  0.049664  0.831052   
                   3  0.966206            0.866436  0.068250  0.795922   
                   4  0.974706            0.930693  0.062414  0.843536   

                               groundspace sharpness            ...  \
                           max                  mean       std  ...   
n Algorithm        p                                            ...   
3 tdvp             1  0.971549              0.294665  0.181596  ...   
                   2  0.982354              0.227546  0.135601  ...   
  scipy            1  0.770741              0.423131  0.003925  ...   
                   2  0.976395              0.271914  0.169845  ...   
  gradient_descent 1  0.971170              0.295927  0.181864  ...   
                   2  0.982388              0.247797  0.164417  ...   
4 tdvp             1  0.962394              0.331865  0.164583  ...   
                   2  0.943499              0.297600  0.061860  ...   
                   3  0.967577              0.324697  0.090627  ...   
                   4  0.977556              0.232359  0.100608  ...   

                     num_fun_calls               steps                      \
                               min     max        mean          std    min   
n Algorithm        p                                                         
3 tdvp             1          56.0    62.0   82.000000     7.071068   77.0   
                   2          74.0   116.0  124.000000    32.526912  101.0   
  scipy            1          23.0    24.0   23.500000     0.707107   23.0   
                   2          48.0    50.0   49.000000     1.414214   48.0   
  gradient_descent 1         148.0   224.0   46.500000    13.435029   37.0   
                   2         528.0  8672.0  575.000000   719.834703   66.0   
4 tdvp             1          44.0    62.0   75.666667     7.312090   62.0   
                   2          56.0   602.0  235.000000   232.458168   79.0   
                   3           0.0  2588.0  774.600000  1247.291987   85.0   
                   4           0.0   182.0  157.000000    45.731098  126.0   

                             path length                                  
                         max        mean       std     

In [7]:
stats['expectation approximation ratio'].reset_index()

,n,Algorithm,p,mean,std,min,max
0,3,tdvp,1,0.911088,0.073714,0.858965,0.963212
1,3,tdvp,2,0.93039,0.063845,0.885245,0.975536
2,3,scipy,1,0.778976,0.112656,0.699316,0.858635
3,3,scipy,2,0.930466,0.058679,0.888974,0.971958
4,3,gradient_descent,1,0.911161,0.073687,0.859056,0.963266
5,3,gradient_descent,2,0.942691,0.04645,0.909846,0.975536
6,4,tdvp,1,0.555697,0.320149,0.133846,0.963225
7,4,tdvp,2,0.914538,0.023949,0.873142,0.947408
8,4,tdvp,3,0.926812,0.027653,0.895857,0.966206
9,4,tdvp,4,0.943432,0.049538,0.869999,0.974706


In [5]:
def algo_num(algo:str)->int:
    assert algo in {'tdvp','scipy','gradient_descent'}
    if algo=='tdvp':
        return 1
    elif algo=='scipy':
        return 0
    elif algo=='gradient_descent':
        return -1

In [6]:
BA_template = dict(
    layout=go.Layout(
    template='plotly_white',
    yaxis=dict(
        linewidth = 1,
        linecolor = 'black',
        gridcolor = 'lightgrey',
        zerolinecolor = 'black',

    ),
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1,        
        range=[-.5,5],
        autorange=True,
        # type='category',
        linewidth = 1,
        linecolor = 'black',
        gridcolor = 'lightgrey',
        # zerolinecolor = 'black',

    ),
    font=dict(
        family='Computer Modern',
        size=11,
    ),
    legend=dict(
        yanchor="bottom",
        y=0.01,
        xanchor="right",
        x=.99,
        
    ),
    # paper_bgcolor='rgba(0,0,0,0)',   # transparent background
    plot_bgcolor='rgba(0,0,0,0)',)
)

In [47]:
yaxis_names = {
    "expectation approximation ratio": "Expectation approximation ratio",
    "groundspace overlap": "Groundspace overlap",
    "groundspace sharpness": "Groundspace sharpness",
    "approximation ratio": "Approximation ratio",
    "circuit count": "Number of circuits",
    "gate count": "Number of gates",
    "num_fun_calls": "Number of function calls",
    "steps": "Number of steps",
}

layout = {
    "expectation approximation ratio": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
    "groundspace overlap": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
    "groundspace sharpness": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
    "approximation ratio": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
    "circuit count": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    type='log',
                    autorange=True,
                ),
                legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=.1,
                )
            ),
    "gate count": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    type='log',
                    autorange=True,
                ),
                legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=.1,
                )
            ),
    "num_fun_calls": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
    "steps": dict(
                template=BA_template,
                xaxis_title="p",
                legend_title_text="Algorithm",
                width=700,
                yaxis=dict(
                    range=[0, 1.1],
                    tickmode="linear",
                    tick0=0,
                    dtick=0.2,
                    autorange=False,
                ),
            ),
}


In [9]:
def plot_results(meas:str,n:int) -> go.Figure:
    df = stats[meas].xs(n, level="n").reset_index()
    # flattend_data
    data_red = pd.concat(
        [
            data["tdvp"].xs(n, level="n").apply(lambda x: x),
            data["scipy"].xs(n, level="n").apply(lambda x: x),
            data["gradient_descent"].xs(n, level="n").apply(lambda x: x),
        ]
    ).reset_index(drop=True)
    data_red

    gridcolor = "lightgrey"

    scatter = px.scatter(
        data_frame=df,
        x=df["p"] + 0.1 * df["Algorithm"].apply(algo_num),  # for the offset
        y="mean",
        error_y="std",
        color="Algorithm",
        symbol="Algorithm",
        # barmode='group',
    )

    box = px.box(
        data_frame=data_red,
        x="p",
        y=meas,
        color="algorithm",
        # points='all',
    )
    box.update_traces(
        quartilemethod="inclusive"
    )  # or "inclusive", or "linear" by default

    fig = scatter

    # rename traces
    newnames = {"tdvp": "TDVP", "scipy": "COBYLA", "gradient_descent": "GD"}
    fig.for_each_trace(
        lambda t: t.update(
            name=newnames.get(t.name, ""),
            legendgroup=newnames.get(t.name, ""),
            hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
        )
    )

    fig.update_layout(
        layout[meas],
        yaxis_title=yaxis_names[meas],
        
    )
    return fig



In [68]:
stats.loc[4,'tdvp',1].index.levels[0]

Index(['approximation ratio', 'circuit count',
       'expectation approximation ratio', 'gate count', 'groundspace overlap',
       'groundspace sharpness', 'num_fun_calls', 'steps', 'path length'],
      dtype='object')

In [124]:
meas = stats.loc[4,'tdvp',1].index.levels[0][7]
n = 5
fig = plot_results(meas, n)
fig.update_layout(
    legend=dict(
        yanchor="bottom",
        xanchor="right",
        y=0.7,
        x=.99,
    ),
)
fig.show()
# fig.write_image(f'plots//{meas}_n_{n}.svg')

In [25]:
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=.1,
    ),
    yaxis=dict(
        type='log',
        )
)
fig.show()

In [42]:
for meas in stats.columns.levels[0]:
    for n in [3, 4, 5]:
        
        fig.write_image(f".//plots//{meas}_n{n}_scatter.svg")


In [ ]:
with open("./results/grid_search_n5_p2_i5.p",'rb') as file:
    grid_search = pickle.load(file)

In [ ]:
with open("results/data.p",'rb') as file:
    data = pickle.load(file)

In [ ]:
idx = (5,1,5)
qaoa = QAOA(data.loc[idx]['instance'],p=idx[1])

In [ ]:
tdvp_path = data.loc[idx]['tdvp']['path']
tdvp_x = np.array([d[0] for d in tdvp_path])
tdvp_y = np.array([d[1] for d in tdvp_path])
tdvp_z = np.array([qaoa.expectation(d) for d in tdvp_path])

In [ ]:
grid_search.max()

In [ ]:
Zlandscape.shape

In [ ]:
griddata((tdvp_x,tdvp_y),tdvp_z,(xi,yi))

In [ ]:

df = grid_search

x = np.array(df.beta)
y = np.array(df.gamma)
z = np.array(df.value)


xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)

X,Y = np.meshgrid(xi,yi)

Zlandscape = griddata((x,y),z,(X,Y), method='cubic')


Zpath_tdvp = griddata((tdvp_x,tdvp_y),tdvp_z,(xi,yi))

fig = go.Figure(
    go.Surface(
        x=xi,
        y=yi,
        z=Zlandscape
        ),
    # go.Scatter3d(
    #     x=tdvp_x,
    #     y=tdvp_y,
    #     z=tdvp_z,
    #     mode='markers'
    # )
    )

fig.update_layout(
    template=BA_template,
    width=1000,
    height=1000
)
fig.show()
fig.write_html('./plots/grid_seach_n5_p1_i5_landscape.html')